# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [30]:
# Write your code below.

%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [31]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [32]:
import os
from glob import glob

price_data=os.getenv("PRICE_DATA")
parquet_files=glob(os.path.join(price_data,'**/*.parquet'), recursive=True)
parquet_files

['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2007\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2007\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2008\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2008\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_200

In [33]:
print(dd.read_parquet(parquet_files).columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source',
       'ticker', 'Year'],
      dtype='object')


In [34]:
print(dd.read_parquet(parquet_files).head())

             Date   Open   High    Low  Close  Adj Close      Volume   source  \
137315 2001-07-19  15.10  15.29  15.00  15.17  11.404394  34994300.0  ACN.csv   
137316 2001-07-20  15.05  15.05  14.80  15.01  11.284108   9238500.0  ACN.csv   
137317 2001-07-23  15.00  15.01  14.55  15.00  11.276587   7501000.0  ACN.csv   
137318 2001-07-24  14.95  14.97  14.70  14.86  11.171341   3537300.0  ACN.csv   
137319 2001-07-25  14.70  14.95  14.65  14.95  11.238999   4208100.0  ACN.csv   

       ticker  Year  
137315    ACN  2001  
137316    ACN  2001  
137317    ACN  2001  
137318    ACN  2001  
137319    ACN  2001  


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [35]:
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

dd_feat = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1), 
        Adj_Close_lag_1 = x['Adj Close'].shift(1),
        returns = (x['Close'] / x['Close'].shift(1)) - 1,
        hi_lo_range = x['High'] - x['Low']
        )
)
dd_feat.head()

C:\Users\kakai\AppData\Local\Temp\ipykernel_14156\1830445064.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_px.groupby('ticker', group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,
ACN,2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,2001,NaN,NaN,NaN,0.29
ACN,2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,2001,15.17,11.404394,-0.010547,0.25
ACN,2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,2001,15.01,11.284108,-0.000666,0.46
ACN,2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,2001,15.00,11.276587,-0.009333,0.27
ACN,2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,2001,14.86,11.171341,0.006057,0.30


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [36]:

pd_feat=dd_feat.compute()

pd_feat['MA_returns_10']=pd_feat.groupby('ticker')['returns'].transform(lambda x: x.rolling(10).mean())
pd_feat.head(20)


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,MA_returns_10
ticker,,,,,,,,,,,,,,
ACN,2016-07-05,113.510002,113.559998,112.629997,113.099998,106.026108,2041100.0,ACN.csv,2016,NaN,NaN,NaN,0.930000,NaN
ACN,2016-07-06,112.629997,113.519997,111.959999,113.519997,106.419830,2667200.0,ACN.csv,2016,113.099998,106.026108,0.003714,1.559998,NaN
ACN,2016-07-07,113.510002,113.900002,112.510002,112.699997,105.651115,2050500.0,ACN.csv,2016,113.519997,106.419830,-0.007223,1.389999,NaN
ACN,2016-07-08,113.730003,115.239998,113.169998,115.110001,107.910400,1962800.0,ACN.csv,2016,112.699997,105.651115,0.021384,2.070000,NaN
ACN,2016-07-11,115.699997,115.980003,115.129997,115.459999,108.238510,1522300.0,ACN.csv,2016,115.110001,107.910400,0.003041,0.850006,NaN
ACN,2016-07-12,115.900002,117.419998,115.599998,116.500000,109.213455,2431400.0,ACN.csv,2016,115.459999,108.238510,0.009007,1.820000,NaN
ACN,2016-07-13,117.000000,117.160004,116.290001,116.830002,109.522820,1537200.0,ACN.csv,2016,116.500000,109.213455,0.002833,0.870003,NaN
ACN,2016-07-14,117.500000,118.000000,116.660004,116.769997,109.466560,1502000.0,ACN.csv,2016,116.830002,109.522820,-0.000514,1.339996,NaN
ACN,2016-07-15,116.480003,116.639999,114.250000,115.089996,107.891647,2749000.0,ACN.csv,2016,116.769997,109.466560,-0.014387,2.389999,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
> No, Dask can compute moving averages, but converting to pandas is simpler and avoids partitioning issues for small datasets.
+ Would it have been better to do it in Dask? Why?
> Yes, if the dataset is very large, because Dask handles out-of-core computation and scales better than pandas.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.